In [6]:
import time
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service


PATH = input('Введите путь к вебдрайверу: ') 
service = Service(PATH)

In [7]:
account_name = input('Введите название интересуещего аккаунта: ').strip()
# проверка на существование аккаунта
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.141 YaBrowser/22.3.2.644'}
resp = requests.get('https://www.tiktok.com/@' + account_name, headers=headers)
while not resp.ok:
    print('Такого аккаунта не существует либо он был введен неверно.\nПопробуйте еще раз')
    account_name = input().strip()
    resp = requests.get('https://www.tiktok.com/@' + account_name, headers=headers)

    
# открываем сайт 
driver = webdriver.Chrome(service=service)
driver.get('https://www.tiktok.com/@' + account_name)

last_height = driver.execute_script("return document.body.scrollHeight")

# for i in range(20): # для ограниченного числа видосов с аккаунта
while True: # для всех видосов с аккаунта
    # Скроллим вниз по странице
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Ждем, пока страницы прогрузится полностью
    time.sleep(2)
    # Считает новую высоту скроллинга (положение страницы) со старой
    new_height = driver.execute_script("return document.body.scrollHeight")
    # если это конец страницы, перестает скроллить 
    if new_height == last_height:
        break
    last_height = new_height

html = driver.page_source
driver.quit()

In [ ]:
# обработаем главную страницу и достанем оттуда ссылки с помощью метода find_all
soup = BeautifulSoup(html, 'lxml')
all_links_list = list(map(lambda x: x.get('href'), soup.find_all('a', {'href': lambda x: x.startswith('https://www.tiktok.com/@')})))

# создадим функцию, которая будет убирать М и К и превращать текст в числовой формат, она будет применена к просмотрам, лайкам, комментариям, репостам
def decode_number(number):
    if 'K' in number:
        return int(float(number.replace('K', '')) * 1_000)
    elif 'M' in number:
        return int(float(number.replace('M', '')) * 1_000_000)
    return int(number)
all_video_views =  list(map(lambda x: decode_number(x.text), soup.find_all('strong', {'data-e2e': 'video-views'})))

def get_all_info(links):
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.141 YaBrowser/22.3.2.644'}
    likes_list, comments_list, reposts_list, videos_text_list = [], [], [], []
     
    # создадим функцию, которая будет доставать необходимую информацию с сайта 
    def get_info_by_link(url):
        resp = requests.get(url, headers=headers)
        resp.encoding = 'utf-8'
        soup = BeautifulSoup(resp.text, 'lxml')

        likes = soup.find('strong', {'data-e2e':"like-count"}).text
        comments = soup.find('strong', {'data-e2e':"comment-count"}).text
        reposts = soup.find('strong', {'data-e2e':"share-count"}).text
        # video_text = re.findall(r'(.+)TikTokUpload', soup.text)[0].strip() 
        video_text = soup.text.split('TikTokUpload')[0].strip() 
        return likes, comments, reposts, video_text

    links_list = []
    video_views = []
    # достанем информацию по всем собранным ссылкам
    for link in links:      
        try:
            info = get_info_by_link(link)
            likes_list.append(info[0])
            comments_list.append(info[1])
            reposts_list.append(info[2])
            videos_text_list.append(info[3])
            links_list.append(link)
            video_views.append(all_video_views[links.index(link)])
        except AttributeError:
            pass
        
    # преобразование в числовой формат
    likes_list = list(map(lambda x: decode_number(x), likes_list))
    comments_list = list(map(lambda x: decode_number(x), comments_list))

    return links_list, likes_list, comments_list, reposts_list, videos_text_list, video_views

links_list, likes_list, comments_list, reposts_list, videos_text_list, video_views = get_all_info(all_links_list)

info = {'link':links_list, 'likes': likes_list, 'views': video_views, 'comments': comments_list, 'reposts': reposts_list, 'video_text': videos_text_list}    
df = pd.DataFrame(info)

In [10]:
filename = input()
df.to_excel(f'{filename}.xlsx')

In [9]:
df

,link,likes,views,comments,reposts,video_text
0,https://www.tiktok.com/@luckashyowa/video/7073...,20700,538700,96,30,Яя не ожидала😅 а вам подапили цветочки??trim.B...
1,https://www.tiktok.com/@luckashyowa/video/7073...,12300,343700,85,54,LIZA💜 (@luckashyowa) on TikTok
2,https://www.tiktok.com/@luckashyowa/video/7073...,59000,1700000,1105,467,Смогли бы снять видео в торговом центре?😁 @_me...
3,https://www.tiktok.com/@luckashyowa/video/7072...,7100,294800,76,13,"Согласны?☺️❤️ С 8 марта, #ЛучшиеЖенщиныМира"
4,https://www.tiktok.com/@luckashyowa/video/7071...,23600,610400,372,101,Блондинки или брюнетки? 🤔@_medkova
...,...,...,...,...,...,...
264,https://www.tiktok.com/@luckashyowa/video/6828...,3584,42800,14,18,мои путешествия этим летом be like:
265,https://www.tiktok.com/@luckashyowa/video/6828...,1014,15200,9,9,Вы не услышите ничего другого больше никогда 😂...
266,https://www.tiktok.com/@luckashyowa/video/6828...,948,12900,5,4,"Если бы на моем месте была Чарли, то это видео..."
267,https://www.tiktok.com/@luckashyowa/video/6828...,2186,29600,11,10,#дуэт с @syava_original МАЖОР я просто МАЖОР🤪 ...
